In [9]:
!pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/ec/6c/fab8113424af5049f85717e8e527ca3773299a3c6b02506e66436e19874f/opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.2/38.8 MB 5.0 MB/s eta 0:00:08
    --------------------------------------- 0.8/38.8 MB 8.1 MB/s eta 0:00:05
   - -------------------------------------- 1.3/38.8 MB 9.2 MB/s eta 0:00:05
   - -------------------------------------- 1.9/38.8 MB 9.9 MB/s eta 0:00:04
   -- ------------------------------------- 2.4/38.8 MB 10.3 MB/s eta 0:00:04
   --- ------------------------------------ 3.0/38.8 MB 10.6 MB/s eta 0:00:04
   --- ------------------------------------ 3.5/38.8 MB 10.7 MB/s eta 0:00:04
   ---- ----------------------------------- 4.1/38.8 MB 10.8 MB/s eta 0:00:04
   ---- ----------------------------------- 4.6/38.8 MB 11.3 MB/s eta 0:00:04
   ----- -

# import

In [ ]:
import cv2
import numpy as np
import os

# 경로 설정

In [ ]:
# 예제 경로 설정 
label_file_path = ""
image_path = ""
output_path = ""

# format 변환 1: 바운딩박스=>행렬

In [3]:
def read_bounding_boxes(label_file_path):
    bounding_boxes = []

    # 파일 읽기
    with open(label_file_path, 'r') as file:
        for line in file:
            # 각 줄에서 첫 번째 숫자(클래스 번호)는 무시하고 나머지 네 개의 숫자를 읽음
            try:
                _, x_center, y_center, width, height = map(float, line.strip().split())
                bounding_boxes.append((x_center, y_center, width, height))  # 튜플로 추가
            except ValueError:
                print(f"라인 '{line.strip()}'을(를) 읽는 중 오류가 발생했습니다.")
    
    # 바운딩 박스가 하나만 있을 경우, 단일 튜플로 반환
    if len(bounding_boxes) == 1:
        return bounding_boxes[0]
    else:
        return np.array(bounding_boxes)

# format 변환 2: YOLO->openCV로 바꾸기

In [4]:
def convert_yolo_to_opencv(bbox, img_width, img_height):
    x_center, y_center, width, height = bbox
    x = int((x_center - width / 2) * img_width)
    y = int((y_center - height / 2) * img_height)
    width = int(width * img_width)
    height = int(height * img_height)
    return (x, y, width, height)

In [30]:
def output_opencvBB(label_file_path, image_path):

    # 이미지 로드 및 크기 확인
    image = cv2.imread(image_path)
    img_height, img_width = image.shape[:2]

    # 바운딩 박스 읽기
    bounding_boxes = read_bounding_boxes(label_file_path)

    # YOLO 형식 바운딩 박스를 OpenCV 형식으로 변환
    opencv_bounding_boxes = []
    if isinstance(bounding_boxes, tuple):
        # 바운딩 박스가 하나만 있을 경우
        opencv_bounding_boxes.append(convert_yolo_to_opencv(bounding_boxes, img_width, img_height))
    else:
        # 여러 바운딩 박스가 있을 경우
        for bbox in bounding_boxes:
            opencv_bounding_boxes.append(convert_yolo_to_opencv(bbox, img_width, img_height))

    opencv_bounding_boxes = opencv_bounding_boxes[0]
    # 결과 출력
    print(opencv_bounding_boxes)
    return opencv_bounding_boxes

(316, 447, 553, 648)


# Grab Cut

image = cv2.imread(image_path) 경로 설정 시 파일명 한글 x

In [31]:
def grabcut():
    # 이미지 로드
    image = cv2.imread(image_path,output_path, opencv_bounding_boxes)

    # 초기 마스크 설정
    mask = np.zeros(image.shape[:2], np.uint8)

    # GrabCut을 위한 임시 배열
    bgd_model = np.zeros((1, 65), np.float64)
    fgd_model = np.zeros((1, 65), np.float64)

    # 바운딩 박스 설정 (x, y, width, height)
    # 예시 바운딩 박스. 필요에 따라 수정해야 합니다.
    rect = opencv_bounding_boxes

    # GrabCut 알고리즘 실행
    cv2.grabCut(image, mask, rect, bgd_model, fgd_model, 5, cv2.GC_INIT_WITH_RECT)

    # 마스크를 통해 확실한 전경과 배경을 설정
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

    # 원본 이미지와 마스크를 사용해 전경 추출
    image_cut = image * mask2[:, :, np.newaxis]

    # 결과 이미지 저장 (RGB 형식을 BGR 형식으로 변환하여 저장)
    cv2.imwrite(output_path, image_cut)

    print(f"GrabCut 결과 이미지가 {output_path}에 저장되었습니다.")

GrabCut 결과 이미지가 C:/Users/rund6/Desktop/Augmentation/gc\grabcut_result5.jpg에 저장되었습니다.
